In [1]:
import qiskit.aqua.operators as opAqua
import qiskit.quantum_info.operators as opTerra

from qiskit import(QuantumCircuit, execute, Aer, ClassicalRegister, IBMQ, aqua)
from qiskit.circuit import *
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer.extensions import snapshot_expectation_value
from qiskit.extensions import HamiltonianGate
from qiskit import QuantumRegister
from qiskit.circuit.library import *
from qiskit.optimization.applications.ising import tsp
from qiskit.tools.visualization import plot_histogram

import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

from itertools import permutations
from time import localtime, strftime, time

backend = Aer.get_backend('qasm_simulator')

C:\Users\geode\anaconda3\lib\site-packages\qiskit\optimization\__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


\section{Définitions des différentes implémentations :}

Algorithme naïf (6.1) :

In [2]:
class DiagOp:
    def __init__(self, liste): self.value = np.array(liste)
    def __add__(self, B): return DiagOp(self.value + B.value)
    def __sub__(self, B): return DiagOp(self.value - B.value)
    def __matmul__(self, B): return DiagOp(self.value * B.value)
    def __mul__(self, n): return DiagOp(n * self.value)
    def __rmul__(self, n): return DiagOp(n * self.value)
    def __invert__(self): return DiagOp(np.conjugate(self.value))
    def __neg__(self): return DiagOp( - self.value)
    def __xor__(self, B):
        if isinstance(B, DiagOp):
            return DiagOp(np.kron(self.value, B.value))
        if isinstance(B, int):
            n = B
            L = [1]
            for k in range(n):
                L = np.kron(L, self.value)
            return DiagOp(L)

    def evolCircuit(self, t):
        circuit = Diagonal(np.exp(- t * 1j * self.value))
        return circuit.copy('gamma = ' + str(t))

    def evolOp(self, t): return opAqua.PrimitiveOp(self.evolCircuit(t).to_gate())

    def expectation(self, state):
        arr = state.to_matrix()
        mod = np.abs(arr) ** 2
        return np.sum(mod * self.value).real
    
def TspQaoaCircuit_naif(num_cities, p, weight, shots = 1000000):
    id = DiagOp([1, 1])
    z = DiagOp([1, -1])

    def bit(i, t): return t * num_cities + i

    def I(): return id ^ N
    def Z(b): return (id ^ b) ^ z ^ (id ^ (N - b - 1))

    def D(i, t):
        b = bit(i, t)
        return 0.5 * (I() - Z(b))

    def UDriver(t):
        circuitUDriver = QuantumCircuit(N)
        circuitUDriver.rx(2 * t, range(N))
        return circuitUDriver


    def buildDiagCost(pen, num_cities, weights):

        # constrainte (a) : chaque ville est visitée une et une seule fois
        H_a = DiagOp([0] * (2 ** N))
        for i in range(num_cities):
            cur = I()
            for t in range(num_cities):
                cur -= D(i, t)
            H_a += cur @ cur

        # constraint (b) : une et une seule ville par étape de temps
        H_b = DiagOp([0] * (2 ** N))
        for t in range(num_cities):
            cur = I()
            for i in range(num_cities):
                cur -= D(i, t)
            H_b += cur @ cur

        # constraint (d) : poids
        H_d = DiagOp([0] * (2 ** N))
        for i in range(num_cities):
            for j in range(num_cities):
                for t in range(num_cities - 1):
                    H_d += D(i, t) @ D(j, t + 1) * weights[i, j] * pen
                H_d += D(i, num_cities - 1) @ D(j, 0) * weights[i, j] * pen

        return H_a + H_b + H_d

    def buildCircuitDriver():
        circuit_driver = QuantumCircuit(N)
        for i in range(N): circuit_driver.x(i)
        return circuit_driver

    def buildQaoaCircuit(betas, gammas):
        qaoaCircuit = QuantumCircuit(N)
        for i in range(N):
            qaoaCircuit.x(i)
            qaoaCircuit.h(i)
        for k in range(p):
            qaoaCircuit.append(H_cost.evolCircuit(gammas[k]), range(N))
            qaoaCircuit.append(UDriver(betas[k]), range(N))
        return qaoaCircuit


    def qaoaCost(params):
        m = len(params) // 2
        betas, gammas = params[:m], params[m:]
        qaoaCircuit = buildQaoaCircuit(betas, gammas)
        state = opAqua.CircuitStateFn(qaoaCircuit)
        return H_cost.expectation(state)


    #weight = np.random.rand(num_cities, num_cities)
    #if symmetric: weight = weight.T @ weight

    N = num_cities ** 2

    penality = 0.1
    H_cost = buildDiagCost(penality, num_cities, weight)

    optParam = opt.minimize(qaoaCost, x0=[.5] * 2 * p)
    betas, gammas = optParam.x[:p], optParam.x[p:]

    circuit = buildQaoaCircuit(betas, gammas)
    circuit.add_register(ClassicalRegister(N))
    circuit.measure(range(N), range(N))
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(circuit, simulator, shots = shots)
    results = job.result()

    return results.get_counts(circuit)

Algorithme optimisé (6.2) :

In [3]:
def TspQaoa_opt(n, p, weightList, shots = 1000000):    
    N = n * (n - 1) // 2
   
    HCost = buildCost(n, weightList)
 
    parBeta, parGamma = param(p)
    circuit = buildQaoaCircuit(n, p, HCost)

    HCostTerra = opTerra.Operator(HCost.to_matrix())
    circuit.snapshot_expectation_value('Cost', HCostTerra, range(N))    
    initX = 2 * np.pi * np.random.random(2 * p) # Génération aléatoire de l'état initial pour la minimisation    
    resultOpt = opt.minimize(qaoaCost, x0 = initX, options = {'maxiter': 20000, 'disp': False}, args = (circuit, p), bounds = [(0, 2 * np.pi)] * 2 * p) 
    listOptBeta, listOptGamma = resultOpt.x[:p], resultOpt.x[p:] # Récupération des paramètres optimaux
    optCircuit = circuit.bind_parameters({parBeta: listOptBeta, parGamma: listOptGamma})    
    optCircuit.measure_all() # Rajout d'un registre classique pour réaliser la mesure
    results = execute(optCircuit, backend, shots = shots).result()
    
    return results.get_counts(optCircuit)

dictI, dictZ, dictXHat = {}, {}, {}
def I(N):
    if not N in dictI:
        dictI[N] = opAqua.I ^ N
    return dictI[N]

def Z(N, b):
    if not (N, b) in dictZ:
        dictZ[(N, b)] = I(N - b - 1) ^ opAqua.Z ^ I(b)
    return dictZ[(N, b)]

def xHat(N, b):
    if not (N, b) in dictZ:
        dictXHat[(N, b)] = 0.5 * (I(N) - Z(N, b))
    return dictXHat[(N, b)]

def buildCost(n, weightList):
    N = n * (n - 1) // 2
    H_1 = 0
    for b in range(N):
        H_1 += weightList[b] * xHat(N, b)

    H_2 = opInvalid(n)
    return H_1 + 2 * np.sum(weightList) * H_2
dictOpInvalid = {}

def opInvalid(n):
    if not n in dictOpInvalid:
        diagonal = []
        N = n * (n - 1) // 2
        for i in range(2 ** N):
            x = np.binary_repr(i, width = N)
            diagonal.append(1 - isValide(x, n)[0])
        dictOpInvalid[n] = opAqua.MatrixOp(np.diag(diagonal))

    return dictOpInvalid[n]

def isValide(x, n):
    N = n * (n - 1) // 2
    tour = []
    Gamma = []
    C = buildC(n)
    for i in range(N):
        if x[i] == '1':
            Gamma.append(C[i])
    if len(Gamma) != n: return 0, x
    S = Gamma.pop().copy()
    k = 0
    while S[k] != 1:
        k +=1
    tour.append(k)
    while 1 in S:
        i = 0
        while i < len(Gamma) and Gamma[i][k] != 1: i += 1
        if i == len(Gamma): return 0, x
        c = Gamma[i]
        for j in range(n):
            if c[j] == 1 and j != k:
                nextk = j
        k = nextk
        tour.append(k)
        S += c
        Gamma.pop(i)
        if max(S) > 2: return 0, x
    if 0 in S: return 0, x
    return 1, tour

def buildC(n):
    C = []
    for i in range(n):
        for j in range(i + 1, n):
            c = np.zeros(n)
            c[i] = 1
            c[j] = 1
            C.append(c)
    return C
dictParam = {}
def param(p): # Crée la liste des paramètres auxquels on assigne des valeurs plus tard
    if not p in dictParam:
        dictParam[p] = (ParameterVector("Beta", p), ParameterVector("Gamma", p))
    return dictParam[p]

# On crée un circuit générique et on assigne les valeurs de gamma et beta plus tard
def buildQaoaCircuit(n, p, HCost):
    N = n * (n - 1) // 2  # N est le nombre d'arêtes du graphe
    qaoaCircuit = QuantumCircuit(N, name = "Global Circuit")

    initCircuit = QuantumCircuit(N, name = 'Init')
    initCircuit.h(range(N))

    qaoaCircuit.append(initCircuit, range(N))
    qaoaCircuit.barrier()


    def UCost(t): # Fonction d'évolution temporelle associée à l'opérateur de coûts
        circuitUCost = QuantumCircuit(N, name = "U_cost")
        circuitUCost.append(HamiltonianGate(HCost, t), range(N))
        return circuitUCost

    def UDriver(t): # Fonction d'évolution temporelle associée à l'opérateur driver 
        circuitUDriver = QuantumCircuit(N, name = "U_driver")
        circuitUDriver.rx(- 2 * t, range(N))
        return circuitUDriver

    parBeta, parGamma = param(p)
    
    for k in range(p): 
        qaoaCircuit.append(UCost(parGamma[k]), range(N))
        qaoaCircuit.append(UDriver(parBeta[k]), range(N))
        qaoaCircuit.barrier()

    return qaoaCircuit

def qaoaCost(parValue, circuit, p):
    listValBeta, listValGamma = parValue[:p], parValue[p:]
    parBeta, parGamma = param(p) 
    
    currentCircuit = circuit.bind_parameters({parBeta: listValBeta, parGamma: listValGamma})

    results = execute(currentCircuit, backend, shots = 1000).result()
    return results.data()['snapshots']['expectation_value']['Cost'][0]['value'].real 

Algorithme générant la superposition des états faisables (7.2) :

In [4]:
def buildNextPathCircuit(n):

    """PARAMETRES"""
    N = n * (n - 1) // 2
    nextN = (n + 1) * n // 2

    edgesToVertex = []
    for i in range(n):
        for j in range(0, i):
            edgesToVertex.append((i, j))

    """CREATION CIRCUIT PRINCIPAL"""
    path = QuantumRegister(nextN, name = "x")
    counter = QuantumRegister(n, name = "n")
    controlPath = QuantumRegister(1, name = "a")
    controlSuperposition = QuantumRegister(1, name = "b")

    circuit = QuantumCircuit(path, controlPath,
                             controlSuperposition,
                             counter,
                             name = "circuit " + str(n) + " -> " +
                             str(n + 1))

    circuit.swap(counter[0], counter[-1])

    circuit.barrier()
    circuit.barrier()

    """BOUCLE GENERALE"""
    for e in range(N):
        j, k = edgesToVertex[e]

        #Incrémentation du compteur
        for ii in range(1, n):
            i = n - ii
            circuit.cswap(path[e], counter[i], counter[i - 1])

        circuit.append(CCXGate(ctrl_state = "10"),
                               [controlSuperposition[0],
                               path[e], controlPath[0]])

        circuit.barrier()

        #Boucle qui change l'arête si besoin en controllant et lui applique la bonne superposition en fonction du compteur
        for i in range(0, n):

            gateSupr = RYGate(2 * np.arcsin(1 / np.sqrt(n - i)))

            ctrlGateSupr = gateSupr.control(2, ctrl_state = "11")
            circuit.append(ctrlGateSupr, [controlPath[0],
                           counter[i], path[e]])

        gateNew = CCXGate(ctrl_state = "01")
        circuit.append(gateNew, [controlPath[0], path[e], path[N + j]])

        circuit.append(gateNew, [controlPath[0], path[e], path[N + k]])

        circuit.barrier()

        circuit.append(CCXGate(ctrl_state = "11"), [path[N + j],
                                path[N + k], controlSuperposition[0]])

        circuit.append(C3XGate(ctrl_state = "110"), [path[e],
                                path[N + j], path[N + k],
                                controlPath[0]])

        circuit.append(CCXGate(ctrl_state = "10"),
                                [controlSuperposition[0],
                                path[e],
                                controlPath[0]])

        circuit.barrier()
        circuit.barrier()

    """REINITIALISATION"""
    circuit.x(controlSuperposition[0])
    circuit.swap(counter[0], counter[-1])

    return circuit

def buildCircuitPath(n):
    N = n * (n - 1) // 2

    path = QuantumRegister(N, name = "x")
    counter = QuantumRegister(n - 1, name = "n")
    controlPath = QuantumRegister(1, name  = "a")
    controlSuperposition = QuantumRegister(1, name = "b")

    #cPath = ClassicalRegister(n * (n - 1) // 2, name = "cX")

    circuit = QuantumCircuit(path, controlPath, controlSuperposition,
                             counter, name = "Superposition")

    """INITIALISATION COMPTEUR + CHEMIN POUR 3"""
    circuit.x(path[:3])
    circuit.x(counter[0])

    circuit.barrier()

    """CONSTRUCTIONS SUCCESSIVES"""
    for i in range(3, n):
        Ni = i * (i + 1) // 2

        circuit.append(buildNextPathCircuit(i), path[:Ni] +
                                                controlPath[:] +
                                                controlSuperposition[:] +
                                                counter[:i])
        circuit.barrier()

    #circuit.measure(path, cPath)

    return circuit

Algorithme final, hamiltonien de conduite $B_1$ (7.3.1) :

In [5]:
def TspQaoaFaisable_B1(n, p, w, shots = 1000000, T = 1):

    #INITIALISATION
    N = n * (n - 1) // 2
    w = w / np.gcd.reduce(w)
    path = QuantumRegister(N, name = "x")
    auxPath = QuantumRegister(n + 1, name = "aux0")
    auxCtrl = QuantumRegister(1, name = "aux1")
    numQubits = N + n + 2

    backend = Aer.get_backend("qasm_simulator")

    qaoaCircuit = QuantumCircuit(path, auxPath, auxCtrl, name = "Global Circuit")
    circuitPath = buildCircuitPath(n)
    inverseCircuitPath = circuitPath.inverse()

    initCircuit = QuantumCircuit(numQubits, name = "Init")
    initCircuit.append(circuitPath, range(numQubits - 1))

    qaoaCircuit.append(initCircuit, range(numQubits))
    qaoaCircuit.barrier()

    #CONTRUCTION DES CIRCUITS UCOST ET UDRIVER
    #Ces circuits sont paramétrés par un paramètre t
    t = Parameter("t")

    UCostCircuit = QuantumCircuit(path, name = "UCost")
    for e in range(N):
        UCostCircuit.p(- t / T * w[e], path[e])

    UDriverCircuit = QuantumCircuit(path, auxPath, auxCtrl, name = "UDriver")
    UDriverCircuit.append(inverseCircuitPath, path[:] + auxPath[:])
    ctrlX = XGate().control(num_ctrl_qubits = N, ctrl_state = "0" * N)
    UDriverCircuit.append(ctrlX, path[:] + auxCtrl[:])
    UDriverCircuit.x(path[0])
    UDriverCircuit.cp(t / T, auxCtrl[0], path[0])
    UDriverCircuit.x(path[0])
    UDriverCircuit.append(ctrlX, path[:] + auxCtrl[:])
    UDriverCircuit.append(circuitPath, path[:] + auxPath[:])

    #CONTRUCTION DU CIRCUIT GLOBAL PARAMETRE
    parBeta = ParameterVector("Beta", p)
    parGamma = ParameterVector("Gamma", p)

    for k in range(p):
        currentUCostCircuit = UCostCircuit.assign_parameters({t: parGamma[k]})
        qaoaCircuit.append(currentUCostCircuit, path)

        currentUDriverCircuit = UDriverCircuit.assign_parameters({t: parBeta[k]})
        #print(currentUDriverCircuit)
        qaoaCircuit.append(currentUDriverCircuit, range(numQubits))

        qaoaCircuit.barrier()

    #CONSTRUCTION DE L'OPERATEUR DE COUT
    I = opAqua.I ^ N
    HCost = 0
    for e in range(N):
        Ze = (opAqua.I ^ (N - e - 1)) ^ opAqua.Z ^ (opAqua.I ^ e)
        HCost += w[e] / 2 * (I - Ze)

    HCostTerra = opTerra.Operator(HCost.to_matrix())
    qaoaCircuit.snapshot_expectation_value('Cost', HCostTerra, path)

    #MINIMISATION
    #Fonction Ã  minimiser
    def qaoaCost(parValue):
        listValBeta, listValGamma = parValue[:p], parValue[p:]

        currentCircuit = qaoaCircuit.bind_parameters({parBeta: listValBeta, parGamma: listValGamma})
        results = execute(currentCircuit, backend, shots = 100).result()
        c = results.data()['snapshots']['expectation_value']['Cost'][0]['value'].real
        return c

    initBeta = np.pi * np.random.random(p)
    initGamma = 2 * np.pi * np.random.random(p)
    initX = np.append(initBeta, initGamma)

    resultOpt = opt.minimize(qaoaCost, x0 = initX, options = {'maxiter': 20000, 'disp': False}, bounds = [(0, np.pi)] * p + [(0, 2 * np.pi)] * p)
    c = qaoaCost(resultOpt.x)
    bestPar = resultOpt.x
    listBeta, listGamma = bestPar[:p], bestPar[p:]

    print(c)

    #EXECUTION FINALE
    pathMeas = ClassicalRegister(N, name = "c")
    bestCircuit = qaoaCircuit.bind_parameters({parBeta: listBeta, parGamma: listGamma})
    bestCircuit.add_register(pathMeas)
    bestCircuit.measure(path, pathMeas)
    #print(bestCircuit)
    results = execute(bestCircuit, backend, shots = shots).result()
    return results.get_counts(bestCircuit)

Algorithme final, hamiltonien de conduite $B_2$ (7.3.2) :

In [6]:
def vertexToEdge(n):
    tab = []
    e = 0
    for i in range(n):
        tab.append([])
        for j in range(0, i):
            tab[i].append(e)
            e += 1
    return tab

def edgesToVertex(n):
    tab = []
    for i in range(n):
        for j in range(0, i):
            tab.append((i, j))
    return tab

def nextPath(l, n):
    if n == 3: return [1, 1, 1]
    prevN = (n - 1) * (n - 2) // 2
    neiN = []
    vte = vertexToEdge(n)
    etv = edgesToVertex(n)

    for i in range(0, n - 1):
        if l[i + prevN] == 1:
            neiN.append(i)

    i, j = neiN
    e0 = vte[j][i]
    l[e0] = 1
    l[i + prevN] = 0
    l[j + prevN] = 0

    e = e0 + 1
    while e < prevN and l[e] == 0:
        e += 1

    if e != (n - 1) * (n - 2) // 2:
        i, j = etv[e]
        l[e] = 0
        l[i + prevN] = 1
        l[j + prevN] = 1

    else:
        l1 = nextPath(l[:prevN], n - 1)

        for i in range(0, prevN):
            l[i] = l1[i]
        e = 0
        while l[e] == 0:
            e += 1
        i, j = etv[e]
        l[e] = 0
        l[i + prevN] = 1
        l[j + prevN] = 1

    return l

def getOnePath(n):
    path = []
    for i in range(1, n):
        for k in range(i - 1):
            path.append(0)
        path.append(1)
    path[(n - 1) * (n - 2) // 2] = 1
    return path

def buildMatrix(n):
    N = n * (n - 1) // 2
    path = getOnePath(n)
    M = np.zeros((2 ** N, 2 ** N))
    nbPath = np.math.factorial(n - 1) // 2

    for i in range(nbPath):
        prevPath = np.copy(path)
        nextPath(path, n)
        k1, k2 = 0, 0
        p2 = 1
        for j in range(N):
            k1 += path[j] * p2
            k2 += prevPath[j] * p2
            p2 *= 2
        M[k1, k2] = 1
        M[k2, k1] = 1

    return M

def TspQaoaFaisable_B2(n, p, w, shots = 1000000, T = 1):

    #INITIALISATION
    N = n * (n - 1) // 2
    w = w / np.gcd.reduce(w)
    path = QuantumRegister(N, name = "x")
    auxPath = QuantumRegister(n + 1, name = "aux0")
    auxCtrl = QuantumRegister(1, name = "aux1")
    numQubits = N + n + 2

    backend = Aer.get_backend("qasm_simulator")

    qaoaCircuit = QuantumCircuit(path, auxPath, auxCtrl, name = "Global Circuit")
    circuitPath = buildCircuitPath(n)
    inverseCircuitPath = circuitPath.inverse()

    initCircuit = QuantumCircuit(numQubits, name = "Init")
    initCircuit.append(circuitPath, range(numQubits - 1))

    qaoaCircuit.append(initCircuit, range(numQubits))
    qaoaCircuit.barrier()

    #CONTRUCTION DES CIRCUITS UCOST ET UDRIVER
    #Ces circuits sont paramÃ©trÃ©s par un paramÃ¨tre t
    t = Parameter("t")

    UCostCircuit = QuantumCircuit(path, name = "UCost")
    for e in range(N):
        UCostCircuit.p(- t / T * w[e], path[e])

    UDriverCircuit = QuantumCircuit(path, auxPath, auxCtrl, name = "UDriver")

    matrixCost = - buildMatrix(n)
    HDriver = opAqua.MatrixOp(matrixCost)
    UDriverOp = (t / T * HDriver).exp_i()
    UDriverCircuit.append(UDriverOp, range(N))

    #CONTRUCTION DU CIRCUIT GLOBAL PARAMETRE
    parBeta = ParameterVector("Beta", p)
    parGamma = ParameterVector("Gamma", p)

    for k in range(p):
        currentUCostCircuit = UCostCircuit.assign_parameters({t: parGamma[k]})
        qaoaCircuit.append(currentUCostCircuit, path)

        currentUDriverCircuit = UDriverCircuit.assign_parameters({t: parBeta[k]})
        qaoaCircuit.append(currentUDriverCircuit, range(numQubits))

        qaoaCircuit.barrier()

    #CONSTRUCTION DE L'OPERATEUR DE COUT
    I = opAqua.I ^ N
    HCost = 0
    for e in range(N):
        Ze = (opAqua.I ^ (N - e - 1)) ^ opAqua.Z ^ (opAqua.I ^ e)
        HCost += w[e] / 2 * (I - Ze)

    HCostTerra = opTerra.Operator(HCost.to_matrix())

    qaoaCircuit.snapshot_expectation_value('Cost', HCostTerra, path)

    #MINIMISATION
    #Fonction Ã  minimiser
    def qaoaCost(parValue):
        listValBeta, listValGamma = parValue[:p], parValue[p:]

        currentCircuit = qaoaCircuit.bind_parameters({parBeta: listValBeta, parGamma: listValGamma})
        results = execute(currentCircuit, backend, shots = 100).result()
        c = results.data()['snapshots']['expectation_value']['Cost'][0]['value'].real
        return c

    initBeta = np.pi * np.random.random(p)
    initGamma = 2 * np.pi * np.random.random(p)
    initX = np.append(initBeta, initGamma)

    resultOpt = opt.minimize(qaoaCost, x0 = initX, options = {'maxiter': 20000, 'disp': False}, bounds = [(0, np.pi)] * p + [(0, 2 * np.pi)] * p)
    c = qaoaCost(resultOpt.x)
    bestPar = resultOpt.x
    listBeta, listGamma = bestPar[:p], bestPar[p:]

    print(c)

    #EXECUTION FINALE
    pathMeas = ClassicalRegister(N, name = "c")
    bestCircuit = qaoaCircuit.bind_parameters({parBeta: listBeta, parGamma: listGamma})
    bestCircuit.add_register(pathMeas)
    bestCircuit.measure(path, pathMeas)
    results = execute(bestCircuit, backend, shots = shots).result()
    return results.get_counts(bestCircuit)

\section{Tests}

In [ ]:
def randWeight(n):
    w = np.random.randint(1, 10, (n, n))
    for i in range(n): w[i, i] = 0
    return w + w.T

def naiveTsp(n, weightMatrix):
    if n == 2: return [0, 1], weightMatrix[0, 1]
    def costTour(tour):
        w = 0
        for i in range(0, n - 1):
            w += weightMatrix[tour[i], tour[i + 1]]
        w += weightMatrix[tour[n - 1], tour[0]]
        return w

    l = []
    for tour in permutations(np.arange(n)):
        w = costTour(tour)
        l.append([w, tour])
    bestCost, bestTour = min(l)
    bestTour = list(bestTour)
    #print(l)
    return bestTour, bestCost

def toStringPath(l, n):
    s = ['0'] * (n * (n - 1) // 2)
    for t in range(n):
        i = max((l[t], l[(t + 1) % n]))
        j = min((l[t], l[(t + 1) % n]))
        s[i * (i - 1) // 2 + j] = '1'
    s.reverse()
    return "".join(s)

def toListWeight(weightMatrix, n):
    weightList = []
    for i in range(n):
        for j in range(i):
            weightList.append(weightMatrix[i, j])
    return np.array(weightList)

T_naif = []
proba_naif = []

T_opt = []
proba_opt = []

T_fin_B1 = []
proba_fin_B1 = []

T_fin_B2 = []
proba_fin_B2 = []

#Choix des critères temporels
T_cd_B = [15, 20, 25, 40]
T_cd_opt = [15, 15, 25, 30]

pMax = 4

for p in range(1, pMax + 1):
    T_naif.append([])
    proba_naif.append([])
    
    T_opt.append([])
    proba_opt.append([])
    
    T_fin_B1.append([])
    proba_fin_B1.append([])
    
    T_fin_B2.append([])
    proba_fin_B2.append([])

nb_villes = 4;
    
for k in range(10):
    wMat = randWeight(nb_villes)
    w = toListWeight(wMat, nb_villes)
    bestPath, _ = naiveTsp(nb_villes, wMat)
    bestString = toStringPath(bestPath, nb_villes)
    
    if nb_villes == 3:
        for p in range(1, 4):
            t = time()
            count_naif = TspQaoaCircuit_naif(nb_villes, p, wMat, shots = 1000000)
            T_naif[p - 1].append(time() - t)
            proba_naif[p - 1].append((count_naif['100010001'] + count_naif['010001100'] + count_naif['001100010'] + count_naif['100001010'] + count_naif['001010100'] + count_naif['010100001'])/ 1000000)
            print(time() - t, (count_naif['100010001'] + count_naif['010001100'] + count_naif['001100010'] + count_naif['100001010'] + count_naif['001010100'] + count_naif['010100001'])/ 1000000)

    for p in range(1, pMax + 1):
        print(k, p)
        
        t = time()
        count_opt = TspQaoa_opt(nb_villes, p, w.tolist(), shots = 1000000)
        while(time()-t < T_cd_opt[p-1]):
            t = time()
            count_opt = TspQaoa_opt(nb_villes, p, w.tolist(), shots = 1000000)       
        T_opt[p - 1].append(time() - t)
        proba_opt[p - 1].append(count_opt[bestString] / 1000000)
        print(time() - t, count_opt[bestString] / 1000000)
        
        t = time()
        count_fin_B1 = TspQaoaFaisable_B1(nb_villes, p, w, shots = 1000000)
        while(time()-t < T_cd_B[p-1]):
            t = time()
            count_fin_B1 = TspQaoaFaisable_B1(nb_villes, p, w, shots = 1000000)
        T_fin_B1[p - 1].append(time() - t)
        proba_fin_B1[p - 1].append(count_fin_B1[bestString] / 1000000)
        print(time() - t, count_fin_B1[bestString] / 1000000)

        t = time()
        count_fin_B2 = TspQaoaFaisable_B2(nb_villes, p, w, shots = 1000000)
        while(time()-t < T_cd_B[p-1]):
            t = time()
            count_fin_B2 = TspQaoaFaisable_B2(nb_villes, p, w, shots = 1000000)
        T_fin_B2[p - 1].append(time() - t)
        proba_fin_B2[p - 1].append(count_fin_B2[bestString] / 1000000)
        print(time() - t, count_fin_B2[bestString] / 1000000)